In [1]:
import yaml
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import traceback
from glob import glob

import sys
sys.path.append("../")

from electronfactors.inputs.convert_merge_guess import convert_merge_guess
from electronfactors.model.initialise import create_cache

from electronfactors.ellipse.equivalent import EquivalentEllipse

from electronfactors.model.iteration import iteration, create_fit

from electronfactors.model.display_cutouts import display_cutouts

In [2]:
# convert_merge_guess()

In [3]:
# energy_list = [6, 9, 12, 15, 18]
# app_list = [6, 10, 14, 20, 25]

# for energy in energy_list:
#     for applicator in app_list:
#         try:
#             create_cache(energy=energy, applicator=applicator)
#         except:
#             print("Energy = %d\nApplicator = %d\n" % (energy, applicator))
#             traceback.print_exc()

In [4]:
model_filepaths = glob("model_cache/*")



In [5]:
# for filepath in model_filepaths:
#     try:
#         iteration(filepath=filepath, n=3)
#     except:
#         print("Energy = %d\nApplicator = %d\n" % (energy, applicator))
#         traceback.print_exc()
    

In [6]:
filepath = model_filepaths[1]

In [7]:
filepath

'model_cache/18MeV_10app_100ssd.yml'

In [9]:
with open(filepath, 'r') as inputFile:
    input_dict = yaml.load(inputFile)

In [12]:
label = [key for key in input_dict]
poi = [input_dict[key]['poi'] for key in label]

width = np.array([input_dict[key]['width'] for key in label])
length = np.array([input_dict[key]['length'] for key in label])
factor = np.array([input_dict[key]['factor'] for key in label])
fit = create_fit(width, length, factor)

In [13]:
# min_radii = np.min(width/2)
min_radii = 0.1  # This is temporary

def circle_fit(radii):
    if radii >= min_radii:
        result = fit(radii*2, 2/radii)
    else:
        result = 0.
    return result

In [ ]:
output_dict = dict()
for i, key in enumerate(label):
    XCoords = input_dict[key]['XCoords']
    YCoords = input_dict[key]['YCoords']

    equivalent_ellipse = EquivalentEllipse(
        x=XCoords, y=YCoords,
        circle_fit=circle_fit, n=3,
        min_distance=min_radii,
        poi=poi[i]
    )

    output_dict[key] = dict()
    output_dict[key]['width'] = equivalent_ellipse.width
    output_dict[key]['length'] = equivalent_ellipse.length

In [ ]:
def predict_factor(fit, width, length):
    ratio = width/length
    eqPonA = 2*(3*(ratio+1) - np.sqrt((3*ratio+1)*(ratio+3))) / width

    return fit(width, eqPonA)

In [ ]:
width_new = np.array([output_dict[key]['width'] for key in label])
length_new = np.array([output_dict[key]['length'] for key in label])

for i, key in enumerate(label):
    width_test = np.delete(width_new, i)
    length_test = np.delete(length_new, i)
    factor_test = np.delete(factor, i)

    fit_test = create_fit(width_test, length_test, factor_test)

    predicted_factor = predict_factor(
        fit_test, width_new[i], length_new[i])

    output_dict[key]['predicted_factor'] = round(
        float(predicted_factor), 4)

for i, key in enumerate(label):
    output_dict[key]['XCoords'] = input_dict[key]['XCoords']
    output_dict[key]['YCoords'] = input_dict[key]['YCoords']
    output_dict[key]['factor'] = input_dict[key]['factor']
    output_dict[key]['energy'] = input_dict[key]['energy']
    output_dict[key]['applicator'] = input_dict[key]['applicator']
    output_dict[key]['ssd'] = input_dict[key]['ssd']
    output_dict[key]['poi'] = input_dict[key]['poi']

    output_dict[key]['width'] = round(
        float(output_dict[key]['width']), 2)
    output_dict[key]['length'] = round(
        float(output_dict[key]['length']), 2)

with open(filepath, 'w') as outfile:
    outfile.write(yaml.dump(output_dict, default_flow_style=False))
